In [1]:
!pip install pandas

     |████████████████████████████████| 10.1MB 3.3MB/s eta 0:00:01
     |████████████████████████████████| 512kB 53.3MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 24.5MB/s eta 0:00:01


In [2]:
import pandas as pd
import numpy as np
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.19.0               |             py_0          53 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          85 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0
  geopy              conda-forge/noarch::geopy-1.19.0-py_0



geographiclib-1.49   | 32 KB     | ##################################### | 100% 
geopy-1.19.0         | 53 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/ninjananjo/Clothing-Store-Competition/master/StoreLocations.csv")
df.shape

(8, 56)

In [4]:
df = df[['TOWN','NAME','ID','COUNTRY','Address1','Postcode','Host']]
df = df[df['ID'].str.startswith("TS")]
df.head()

,TOWN,NAME,ID,COUNTRY,Address1,Postcode,Host
2,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TS0001,ENG,214 Oxford Street,W1W 8LG,TS0001
3,LONDON (CITY),CHEAPSIDE,TS0873,ENG,Unit Msu1,EC2V 6AH,TS0873
6,MANCHESTER (TRAFFORD C),TRAFFORD CENTRE,TS1012,ENG,130-134 Regent Crescent,M17 8AF,TS1012


In [5]:
#Add lontitude and lattitude data
addresses = df.loc[:,'Postcode']
addresses_alt = df.loc[:,'NAME']
latitude = []
longitude = []

for address, address_alt in zip(addresses, addresses_alt):
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address+", UK")
    location_alt = geolocator.geocode(address_alt+", UK")
    try:
        #print('a')
        latitude.append(location.latitude)
        longitude.append(location.longitude)
        
    
    except:
        
        try:
            #print('b')
            latitude.append(location_alt.latitude)
            longitude.append(location_alt.longitude)
            
        except:    
            #print('c')
            latitude.append(0)
            longitude.append(0)
            
        
df.loc[:,'Latitude']= latitude
df.loc[:,'Longitude']= longitude

In [6]:
#remove regions that location information was not available
df = df[(df['Latitude']!=0)| (df['Longitude']!=0)]
print(df.shape[0])
df

3


,TOWN,NAME,ID,COUNTRY,Address1,Postcode,Host,Latitude,Longitude
2,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TS0001,ENG,214 Oxford Street,W1W 8LG,TS0001,51.515534,-0.141352
3,LONDON (CITY),CHEAPSIDE,TS0873,ENG,Unit Msu1,EC2V 6AH,TS0873,51.514098,-0.094914
6,MANCHESTER (TRAFFORD C),TRAFFORD CENTRE,TS1012,ENG,130-134 Regent Crescent,M17 8AF,TS1012,53.462559,-2.346119


In [7]:
CLIENT_ID = 'QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO' # your Foursquare ID
CLIENT_SECRET = '1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH' # your Foursquare Secret
VERSION = '20190501' # Foursquare API version
LIMIT = 400 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius
categoryId = '4bf58dd8d48988d1f2941735'

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    
    region_list = []
    
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(    
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
        
        #print(url) 
        
        # make the GET request
        results = requests.get(url).json()["response"]
        
        #flatten JSON file
        resultsVen = json_normalize(results['venues'])
        resultsCat = json_normalize(data=results['venues'], record_path=['categories'], record_prefix='cat_', errors='ignore')
        resultsVenCat = pd.concat([resultsVen, resultsCat], axis=1)
        resultsVenCat = resultsVenCat[['id','name','location.distance','location.formattedAddress','location.lat','location.lng','cat_name']]
        
        #Add region name and average price 
        for i in range(0,resultsVen.shape[0]):
            region_list.append(name)
                    
        region_list = pd.DataFrame(region_list, columns=['Store ID'])
        
        nearbyVenues = pd.concat([region_list, resultsVenCat], axis=1)

    return(nearbyVenues)

In [9]:
regions_venues = pd.DataFrame()

#request venue information for each region and store in regions_venues
for region in df['ID'].unique():
    region_venues = df[df['ID']==region]
    regions_venues = regions_venues.append((getNearbyVenues(names=region_venues['ID'],
                                                  latitudes=region_venues['Latitude'],
                                                  longitudes=region_venues['Longitude'],
                                                  radius=radius)),ignore_index=True)
    
#rename columns
regions_venues.rename(columns={'id':'Venue ID',
                          'name':'Venue Name',
                          'location.distance':'Venue Distance',
                          'location.formattedAddress':'Venue Address',
                          'location.lat':'Venue Lat',
                          'location.lng':'Venue Lng',
                          'cat_name':'Venue Category'}, inplace=True)

print(regions_venues.shape[0])

113


In [12]:
jdsports = regions_venues[regions_venues['Venue Name']=='JD Sports']
jdsports.to_csv('jdsports_proximty.csv')
jdsports

,Store ID,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
2,TS0001,4ac518f1f964a520ecad20e3,JD Sports,127,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.139567,Sporting Goods Shop
13,TS0001,5895ef8201f0776e38be9d23,JD Sports,579,"[385-389 Oxford St, Mayfair, Greater London, U...",51.514111,-0.149394,Sporting Goods Shop
58,TS0873,4ccc1f952dc4370440aac408,JD Sports,2331,"[55 Chapel Street Market, Islington, Greater L...",51.533512,-0.107551,Sporting Goods Shop
95,TS1012,4edd380cbe7be2833b142d45,JD Sports,482,"[48 Regents Crescent, Unit L12, Trafford Centr...",53.466613,-2.348693,Sporting Goods Shop


In [ ]:
#Which TM/TS stores do not have a JD Sports withing a 2km radius?
#Add map feature with TM/TS as centre point and surrounding JD Sports.